# Applying Continuum Properties to Pores

When attempting to model materials with pore sizes that span multiple length scales, it can become computationally infeasible to resolve all the pores at all scales.  Often this is not really necessary since the transport on the smaller scales can be effectively treated as occuring in a continua with minimal heterogeneity.  This example will illustrate how to assign continuum properties to some nodes in the network, while retaining the pore-scale resolution elsewhere.

In [1]:
import openpnm as op
import matplotlib.pyplot as plt
ws = op.Workspace()
ws.settings['log_level'] = 50

First we need to create our network:

In [2]:
pn = op.network.Cubic(shape=[10, 10, 1], spacing=1)

Let's assume that the left half of the network will retain pore network properties, and the right half will be treated as a continua.  For convenience, let's label the pores and throats accordingly.  

| Further Reading |
|:---|
| The use of labels is explained in [this tutorial](../tutorials/using_and_creating_labels.ipynb) |


In [3]:
Ps = pn['pore.coords'][:, 0] <= 5
Ts = pn.find_neighbor_throats(pores=Ps, mode='or')
pn.set_label(label='pnm', pores=Ps, throats=Ts)

This simulation will require creating two subdomains, one for the PNM pores and another for the continuum pores.  

| Further Reading |
|:---|
| The process of creating and using subdomains is outline in [this tutorial](../tutorials/using_and_creating_labels.ipynb) |


In [5]:
geo_pnm = op.geometry.StickAndBall(network=pn, pores=pn.pores('pnm'), throats=pn.throats('pnm'))
geo_con = op.geometry.GenericGeometry(network=pn, pores=pn.pores('pnm', mode='not'), throats=pn.throats('pnm', mode='not'))

In [ ]:
class Continua(GenericGeometry):
    def __init__(self, porosity, **kwargs):
        self['pore.porosity'] = porosity
        